## **Library Import**

In [791]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import requests
from dotenv import load_dotenv
import pickle

## **Setup**

In [792]:
load_dotenv()

True

In [793]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6292240826025855916
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1734816564
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8795916528476170044
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


In [794]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

## **Data Import**

In [795]:
development = os.getenv('BACKEND_URL')
headers = {
  'Authorization': 'Bearer ' + os.getenv('DB_TOKEN'),
  'Cache-Control': 'no-cache',
  'Connection': 'keep-alive',
}

### Farmer

In [796]:
farmer_url = development + '/farmers'

response = requests.get(farmer_url, headers=headers)

if response.status_code == 200:
    farmer_data = response.json().get('data')
    farmer_df = pd.DataFrame(farmer_data)
else:
    print('Error!')

farmer_df = farmer_df[['id', 'nama', 'jenis_sawah']]
farmer_df

,id,nama,jenis_sawah
0,1000a253-a43c-4ab3-91f7-8c370e01b9c4,Syawalludin,Dataran Tinggi
1,2345a62a-8d3a-4f7b-aad5-7e50670a817b,Auva,Irigasi
2,58c5ee91-37b5-4818-8815-872eae86f21c,Budiman,Dataran Tinggi
3,7488ab52-bc5b-4ff2-9427-a01b64145c6c,Suryadi,Dataran Rendah
4,a5d3b9fe-eb1f-4c7e-84fb-fd95a39697a7,Kevin,Ladang
5,cecb6e32-8f30-4806-a16c-a6073966ad65,Amriz,Ladang
6,e9ef08e2-9d2d-49d4-8934-7f259cbbf867,Asepuri,Dataran Rendah


### Rating

In [797]:
rating_url = development + '/rating/farmer'

response = requests.get(rating_url, headers=headers)

if response.status_code == 200:
  rating_data = response.json().get('data')
  rating_df = pd.DataFrame(rating_data)
else:
  print('Error!')

rating_df = rating_df[['id', 'farmer_id', 'user_id', 'rating']]
rating_df

,id,farmer_id,user_id,rating
0,726caa63-6ee4-417e-b7d1-454a497f6c9f,a5d3b9fe-eb1f-4c7e-84fb-fd95a39697a7,b0af16ed-0633-42a2-afc7-2c5e48db5a48,4
1,a2fab6fd-6b48-4b53-9fac-e9b4f16b29b5,e9ef08e2-9d2d-49d4-8934-7f259cbbf867,c74c14bf-d6ae-455d-842c-6c02cd638349,2
2,a4488534-1e93-4ac6-b747-70e30cdb84ea,7488ab52-bc5b-4ff2-9427-a01b64145c6c,c74c14bf-d6ae-455d-842c-6c02cd638349,3
3,b31ceeb8-68c8-4828-af93-4937165fbabb,58c5ee91-37b5-4818-8815-872eae86f21c,2bd66dd6-5ac6-40ad-968e-1271da13db32,5
4,d63ba28f-abd3-4614-a28e-5cea2a6d554a,2345a62a-8d3a-4f7b-aad5-7e50670a817b,b0af16ed-0633-42a2-afc7-2c5e48db5a48,4
5,e8e358f0-dc2c-4070-a050-dd8882d2a6ce,cecb6e32-8f30-4806-a16c-a6073966ad65,c74c14bf-d6ae-455d-842c-6c02cd638349,5
6,fab931c1-6189-479c-8c8a-53e65f8cd06d,1000a253-a43c-4ab3-91f7-8c370e01b9c4,2bd66dd6-5ac6-40ad-968e-1271da13db32,4


## **Data Preprocessing**

In [798]:
data = pd.merge(farmer_df, rating_df, left_on='id', right_on='farmer_id', how='left')
data = data[['id_x', 'nama', 'jenis_sawah', 'user_id', 'rating']]
data = data.rename(columns={'id_x': 'farmer_id'})
data

,farmer_id,nama,jenis_sawah,user_id,rating
0,1000a253-a43c-4ab3-91f7-8c370e01b9c4,Syawalludin,Dataran Tinggi,2bd66dd6-5ac6-40ad-968e-1271da13db32,4
1,2345a62a-8d3a-4f7b-aad5-7e50670a817b,Auva,Irigasi,b0af16ed-0633-42a2-afc7-2c5e48db5a48,4
2,58c5ee91-37b5-4818-8815-872eae86f21c,Budiman,Dataran Tinggi,2bd66dd6-5ac6-40ad-968e-1271da13db32,5
3,7488ab52-bc5b-4ff2-9427-a01b64145c6c,Suryadi,Dataran Rendah,c74c14bf-d6ae-455d-842c-6c02cd638349,3
4,a5d3b9fe-eb1f-4c7e-84fb-fd95a39697a7,Kevin,Ladang,b0af16ed-0633-42a2-afc7-2c5e48db5a48,4
5,cecb6e32-8f30-4806-a16c-a6073966ad65,Amriz,Ladang,c74c14bf-d6ae-455d-842c-6c02cd638349,5
6,e9ef08e2-9d2d-49d4-8934-7f259cbbf867,Asepuri,Dataran Rendah,c74c14bf-d6ae-455d-842c-6c02cd638349,2


## **Content-Based Filtering**

In [799]:
cb_df = data[['farmer_id', 'nama', 'jenis_sawah']].copy()
cb_df.drop_duplicates(subset=['farmer_id'], inplace=True)
cb_df['jenis_sawah'] = cb_df['jenis_sawah'].apply(lambda x: x.lower()).str.replace(' ', '')
cb_df

,farmer_id,nama,jenis_sawah
0,1000a253-a43c-4ab3-91f7-8c370e01b9c4,Syawalludin,datarantinggi
1,2345a62a-8d3a-4f7b-aad5-7e50670a817b,Auva,irigasi
2,58c5ee91-37b5-4818-8815-872eae86f21c,Budiman,datarantinggi
3,7488ab52-bc5b-4ff2-9427-a01b64145c6c,Suryadi,dataranrendah
4,a5d3b9fe-eb1f-4c7e-84fb-fd95a39697a7,Kevin,ladang
5,cecb6e32-8f30-4806-a16c-a6073966ad65,Amriz,ladang
6,e9ef08e2-9d2d-49d4-8934-7f259cbbf867,Asepuri,dataranrendah


In [800]:
tfid = TfidfVectorizer()

tfid.fit(cb_df['jenis_sawah'])

tfid.get_feature_names_out()

array(['dataranrendah', 'datarantinggi', 'irigasi', 'ladang'],
      dtype=object)

In [801]:
tfidf_vectors = tfid.transform(cb_df['jenis_sawah'])

print(tfidf_vectors.shape)

(7, 4)


In [802]:
tfidf_matrix = tfidf_vectors.todense()
tfidf_matrix

matrix([[0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.]])

In [803]:
pd.DataFrame(tfidf_matrix, columns=tfid.get_feature_names_out(), index=cb_df['farmer_id'])

,dataranrendah,datarantinggi,irigasi,ladang
farmer_id,,,,
1000a253-a43c-4ab3-91f7-8c370e01b9c4,0.0,1.0,0.0,0.0
2345a62a-8d3a-4f7b-aad5-7e50670a817b,0.0,0.0,1.0,0.0
58c5ee91-37b5-4818-8815-872eae86f21c,0.0,1.0,0.0,0.0
7488ab52-bc5b-4ff2-9427-a01b64145c6c,1.0,0.0,0.0,0.0
a5d3b9fe-eb1f-4c7e-84fb-fd95a39697a7,0.0,0.0,0.0,1.0
cecb6e32-8f30-4806-a16c-a6073966ad65,0.0,0.0,0.0,1.0
e9ef08e2-9d2d-49d4-8934-7f259cbbf867,1.0,0.0,0.0,0.0


In [804]:
cosine_sim = cosine_similarity(tfidf_vectors)
cosine_sim

array([[1., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 1.],
       [0., 0., 0., 0., 1., 1., 0.],
       [0., 0., 0., 0., 1., 1., 0.],
       [0., 0., 0., 1., 0., 0., 1.]])

In [805]:
cosine_sim_df = pd.DataFrame(cosine_sim, columns=cb_df['farmer_id'], index=cb_df['farmer_id'])

print(cosine_sim_df.shape)

cosine_sim_df

(7, 7)


farmer_id,1000a253-a43c-4ab3-91f7-8c370e01b9c4,2345a62a-8d3a-4f7b-aad5-7e50670a817b,58c5ee91-37b5-4818-8815-872eae86f21c,7488ab52-bc5b-4ff2-9427-a01b64145c6c,a5d3b9fe-eb1f-4c7e-84fb-fd95a39697a7,cecb6e32-8f30-4806-a16c-a6073966ad65,e9ef08e2-9d2d-49d4-8934-7f259cbbf867
farmer_id,,,,,,,
1000a253-a43c-4ab3-91f7-8c370e01b9c4,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2345a62a-8d3a-4f7b-aad5-7e50670a817b,0.0,1.0,0.0,0.0,0.0,0.0,0.0
58c5ee91-37b5-4818-8815-872eae86f21c,1.0,0.0,1.0,0.0,0.0,0.0,0.0
7488ab52-bc5b-4ff2-9427-a01b64145c6c,0.0,0.0,0.0,1.0,0.0,0.0,1.0
a5d3b9fe-eb1f-4c7e-84fb-fd95a39697a7,0.0,0.0,0.0,0.0,1.0,1.0,0.0
cecb6e32-8f30-4806-a16c-a6073966ad65,0.0,0.0,0.0,0.0,1.0,1.0,0.0
e9ef08e2-9d2d-49d4-8934-7f259cbbf867,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [806]:
def get_cb_recommendations(farmer_id, similarity_data=cosine_sim_df, items=cb_df[['farmer_id' ,'nama', 'jenis_sawah']], k=5):
  index = similarity_data.loc[farmer_id].to_numpy().argpartition(
      range(-1, -k, -1))
  
  closest = similarity_data.columns[index[-1:-(k+2):-1]]

  closest = closest.drop(farmer_id, errors='ignore')

  return pd.DataFrame(closest).merge(items).head(k)

In [807]:
get_cb_recommendations('1000a253-a43c-4ab3-91f7-8c370e01b9c4')

,farmer_id,nama,jenis_sawah
0,58c5ee91-37b5-4818-8815-872eae86f21c,Budiman,datarantinggi
1,2345a62a-8d3a-4f7b-aad5-7e50670a817b,Auva,irigasi
2,e9ef08e2-9d2d-49d4-8934-7f259cbbf867,Asepuri,dataranrendah
3,cecb6e32-8f30-4806-a16c-a6073966ad65,Amriz,ladang
4,7488ab52-bc5b-4ff2-9427-a01b64145c6c,Suryadi,dataranrendah


## **Collaborative Filtering**

In [808]:
collab_df = data[['user_id', 'farmer_id', 'rating']].copy()
collab_df.dropna(inplace=True)

In [809]:
user_id = collab_df['user_id'].unique().tolist()
print("jumlah user yang memberikan rating: ", len(user_id))

user_id_encoded = {x: i for i, x in enumerate(user_id)}
list(user_id_encoded.items())[:5]

jumlah user yang memberikan rating:  3


[('2bd66dd6-5ac6-40ad-968e-1271da13db32', 0),
 ('b0af16ed-0633-42a2-afc7-2c5e48db5a48', 1),
 ('c74c14bf-d6ae-455d-842c-6c02cd638349', 2)]

In [810]:
farmer_id = collab_df['farmer_id'].unique().tolist()
print("jumlah farmer yang diberi rating: ", len(farmer_id))

farmer_id_encoded = {x: i for i, x in enumerate(farmer_id)}
list(farmer_id_encoded.items())[:5]

jumlah farmer yang diberi rating:  7


[('1000a253-a43c-4ab3-91f7-8c370e01b9c4', 0),
 ('2345a62a-8d3a-4f7b-aad5-7e50670a817b', 1),
 ('58c5ee91-37b5-4818-8815-872eae86f21c', 2),
 ('7488ab52-bc5b-4ff2-9427-a01b64145c6c', 3),
 ('a5d3b9fe-eb1f-4c7e-84fb-fd95a39697a7', 4)]

In [811]:
collab_df['user'] = collab_df['user_id'].map(user_id_encoded)
collab_df['farmer'] = collab_df['farmer_id'].map(farmer_id_encoded)

collab_df

,user_id,farmer_id,rating,user,farmer
0,2bd66dd6-5ac6-40ad-968e-1271da13db32,1000a253-a43c-4ab3-91f7-8c370e01b9c4,4,0,0
1,b0af16ed-0633-42a2-afc7-2c5e48db5a48,2345a62a-8d3a-4f7b-aad5-7e50670a817b,4,1,1
2,2bd66dd6-5ac6-40ad-968e-1271da13db32,58c5ee91-37b5-4818-8815-872eae86f21c,5,0,2
3,c74c14bf-d6ae-455d-842c-6c02cd638349,7488ab52-bc5b-4ff2-9427-a01b64145c6c,3,2,3
4,b0af16ed-0633-42a2-afc7-2c5e48db5a48,a5d3b9fe-eb1f-4c7e-84fb-fd95a39697a7,4,1,4
5,c74c14bf-d6ae-455d-842c-6c02cd638349,cecb6e32-8f30-4806-a16c-a6073966ad65,5,2,5
6,c74c14bf-d6ae-455d-842c-6c02cd638349,e9ef08e2-9d2d-49d4-8934-7f259cbbf867,2,2,6


In [812]:
collab_df = collab_df.sample(frac=1, random_state=42)
collab_df.head()

,user_id,farmer_id,rating,user,farmer
0,2bd66dd6-5ac6-40ad-968e-1271da13db32,1000a253-a43c-4ab3-91f7-8c370e01b9c4,4,0,0
1,b0af16ed-0633-42a2-afc7-2c5e48db5a48,2345a62a-8d3a-4f7b-aad5-7e50670a817b,4,1,1
5,c74c14bf-d6ae-455d-842c-6c02cd638349,cecb6e32-8f30-4806-a16c-a6073966ad65,5,2,5
2,2bd66dd6-5ac6-40ad-968e-1271da13db32,58c5ee91-37b5-4818-8815-872eae86f21c,5,0,2
4,b0af16ed-0633-42a2-afc7-2c5e48db5a48,a5d3b9fe-eb1f-4c7e-84fb-fd95a39697a7,4,1,4


In [813]:
x = collab_df[['user', 'farmer']].values

max_rating = collab_df['rating'].max()
min_rating = collab_df['rating'].min()

y = collab_df['rating'].apply(lambda x: (
  x - min_rating) / (max_rating - min_rating)).values

train_indices = int(0.8 * collab_df.shape[0])

x_train, x_val, y_train, y_val = (
  x[:train_indices],
  x[train_indices:],
  y[:train_indices],
  y[train_indices:]
)

print(x, y)
print("Jumlah data train: ", len(x_train))
print("Jumlah data val: ", len(x_val))

[[0 0]
 [1 1]
 [2 5]
 [0 2]
 [1 4]
 [2 3]
 [2 6]] [0.66666667 0.66666667 1.         1.         0.66666667 0.33333333
 0.        ]
Jumlah data train:  5
Jumlah data val:  2


In [814]:
class RecommenderNet(keras.Model):
  def __init__(self, num_users, num_farmer, embedding_size, **kwargs):
    super(RecommenderNet, self).__init__(**kwargs)
    self.num_users = num_users
    self.num_farmer = num_farmer
    self.embedding_size = embedding_size
    self.user_embedding = layers.Embedding(
        num_users,
        embedding_size,
        embeddings_initializer="he_normal",
        embeddings_regularizer=keras.regularizers.l2(1e-6),
    )
    self.user_bias = layers.Embedding(num_users, 1)
    self.farmer_embedding = layers.Embedding(
        num_farmer,
        embedding_size,
        embeddings_initializer="he_normal",
        embeddings_regularizer=keras.regularizers.l2(1e-6),
    )
    self.farmer_bias = layers.Embedding(num_farmer, 1)

  def call(self, inputs):
    user_vector = self.user_embedding(inputs[:, 0])
    user_bias = self.user_bias(inputs[:, 0])
    farmer_vector = self.farmer_embedding(inputs[:, 1])
    farmer_bias = self.farmer_bias(inputs[:, 1])
    dot_user_farmer = tf.tensordot(user_vector, farmer_vector, 2)
    x = dot_user_farmer + user_bias + farmer_bias
    return tf.nn.sigmoid(x)
  
model = RecommenderNet(num_users=len(user_id_encoded), num_farmer=len(farmer_id_encoded), embedding_size=100)

model.build(input_shape=(None, 2))

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(), 
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

model.summary()

Model: "recommender_net_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_88 (Embedding)    multiple                  300       
                                                                 
 embedding_89 (Embedding)    multiple                  3         
                                                                 
 embedding_90 (Embedding)    multiple                  700       
                                                                 
 embedding_91 (Embedding)    multiple                  7         
                                                                 
Total params: 1,010
Trainable params: 1,010
Non-trainable params: 0
_________________________________________________________________


In [815]:
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=64,
    epochs=5,
    verbose=1,
    validation_data=(x_val, y_val),
)

Epoch 1/5


1/1 [==============================] - 1s 604ms/step - loss: 0.9447 - root_mean_squared_error: 0.2511 - val_loss: 2.6256 - val_root_mean_squared_error: 0.8075
Epoch 2/5
1/1 [==============================] - 0s 30ms/step - loss: 0.8499 - root_mean_squared_error: 0.2466 - val_loss: 2.6329 - val_root_mean_squared_error: 0.8079
Epoch 3/5
1/1 [==============================] - 0s 31ms/step - loss: 0.7590 - root_mean_squared_error: 0.2394 - val_loss: 2.6398 - val_root_mean_squared_error: 0.8082
Epoch 4/5
1/1 [==============================] - 0s 29ms/step - loss: 0.6747 - root_mean_squared_error: 0.2283 - val_loss: 2.6463 - val_root_mean_squared_error: 0.8085
Epoch 5/5
1/1 [==============================] - 0s 31ms/step - loss: 0.6009 - root_mean_squared_error: 0.2119 - val_loss: 2.6524 - val_root_mean_squared_error: 0.8088


In [816]:
def get_collab_recommendations(user_id, items=cb_df[['farmer_id', 'nama', 'jenis_sawah']], k=5):
  # ambil data user yang sudah memberikan rating
  farmer_reviewed_by_user = collab_df[collab_df['user_id'] == user_id]

  farmer_not_reviewed = items[~items['farmer_id'].isin(  # ambil data user yang belum memberikan rating
    farmer_reviewed_by_user['farmer_id'])]

  farmer_not_reviewed = list(
    set(farmer_not_reviewed['farmer_id'].tolist()).intersection(set(farmer_id_encoded.keys())))  # ambil data user yang belum memberikan rating

  farmer_not_reviewed = [[user_id_encoded.get(user_id), farmer_id_encoded.get(
    farmer_id)] for farmer_id in farmer_not_reviewed]  # encode user dan farmer

  user_encoder = user_id_encoded.get(user_id)  # encode user

  user_farmer_array = np.hstack(
    ([[user_encoder]] * len(farmer_not_reviewed), farmer_not_reviewed))  # gabungkan user dan farmer

  ratings = model.predict(user_farmer_array).flatten()  # prediksi rating

  top_ratings_indices = ratings.argsort()[-k:][::-1]  # ambil 5 teratas

  farmer_encoded = {i: x for i, x in enumerate(farmer_id_encoded)}

  recommended_farmer_ids = [farmer_encoded.get(
      farmer_not_reviewed[x][0]) for x in top_ratings_indices]  # decode farmer
  
  recommended_farmer = items[items['farmer_id'].isin(recommended_farmer_ids)]

  return recommended_farmer

In [817]:
get_collab_recommendations(user_id='c74c14bf-d6ae-455d-842c-6c02cd638349', k=10)

1/1 [==============================] - 0s 75ms/step


,farmer_id,nama,jenis_sawah
2,58c5ee91-37b5-4818-8815-872eae86f21c,Budiman,datarantinggi


## **Saved Model**

In [818]:
folder = './model'
if not os.path.exists(folder):
  os.makedirs(folder)

#### Content-Based Filtering

In [819]:
pickle.dump(cb_df, open(folder + '/cb_df.pkl', 'wb'))
pickle.dump(cosine_sim_df, open(folder + '/cosine_sim_df.pkl', 'wb'))

In [820]:
pickle.load(open(folder + '/cb_df.pkl', 'rb'))

,farmer_id,nama,jenis_sawah
0,1000a253-a43c-4ab3-91f7-8c370e01b9c4,Syawalludin,datarantinggi
1,2345a62a-8d3a-4f7b-aad5-7e50670a817b,Auva,irigasi
2,58c5ee91-37b5-4818-8815-872eae86f21c,Budiman,datarantinggi
3,7488ab52-bc5b-4ff2-9427-a01b64145c6c,Suryadi,dataranrendah
4,a5d3b9fe-eb1f-4c7e-84fb-fd95a39697a7,Kevin,ladang
5,cecb6e32-8f30-4806-a16c-a6073966ad65,Amriz,ladang
6,e9ef08e2-9d2d-49d4-8934-7f259cbbf867,Asepuri,dataranrendah


In [821]:
pickle.load(open(folder + '/cosine_sim_df.pkl', 'rb'))

farmer_id,1000a253-a43c-4ab3-91f7-8c370e01b9c4,2345a62a-8d3a-4f7b-aad5-7e50670a817b,58c5ee91-37b5-4818-8815-872eae86f21c,7488ab52-bc5b-4ff2-9427-a01b64145c6c,a5d3b9fe-eb1f-4c7e-84fb-fd95a39697a7,cecb6e32-8f30-4806-a16c-a6073966ad65,e9ef08e2-9d2d-49d4-8934-7f259cbbf867
farmer_id,,,,,,,
1000a253-a43c-4ab3-91f7-8c370e01b9c4,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2345a62a-8d3a-4f7b-aad5-7e50670a817b,0.0,1.0,0.0,0.0,0.0,0.0,0.0
58c5ee91-37b5-4818-8815-872eae86f21c,1.0,0.0,1.0,0.0,0.0,0.0,0.0
7488ab52-bc5b-4ff2-9427-a01b64145c6c,0.0,0.0,0.0,1.0,0.0,0.0,1.0
a5d3b9fe-eb1f-4c7e-84fb-fd95a39697a7,0.0,0.0,0.0,0.0,1.0,1.0,0.0
cecb6e32-8f30-4806-a16c-a6073966ad65,0.0,0.0,0.0,0.0,1.0,1.0,0.0
e9ef08e2-9d2d-49d4-8934-7f259cbbf867,0.0,0.0,0.0,1.0,0.0,0.0,1.0


#### Collaborative Filtering

In [822]:
pickle.dump(collab_df, open(folder + '/collab_df.pkl', 'wb'))
pickle.dump(user_id_encoded, open(folder + '/user_id_encoded.pkl', 'wb'))
pickle.dump(farmer_id_encoded, open(folder + '/farmer_id_encoded.pkl', 'wb'))

In [823]:
pickle.load(open(folder + '/collab_df.pkl', 'rb'))

,user_id,farmer_id,rating,user,farmer
0,2bd66dd6-5ac6-40ad-968e-1271da13db32,1000a253-a43c-4ab3-91f7-8c370e01b9c4,4,0,0
1,b0af16ed-0633-42a2-afc7-2c5e48db5a48,2345a62a-8d3a-4f7b-aad5-7e50670a817b,4,1,1
5,c74c14bf-d6ae-455d-842c-6c02cd638349,cecb6e32-8f30-4806-a16c-a6073966ad65,5,2,5
2,2bd66dd6-5ac6-40ad-968e-1271da13db32,58c5ee91-37b5-4818-8815-872eae86f21c,5,0,2
4,b0af16ed-0633-42a2-afc7-2c5e48db5a48,a5d3b9fe-eb1f-4c7e-84fb-fd95a39697a7,4,1,4
3,c74c14bf-d6ae-455d-842c-6c02cd638349,7488ab52-bc5b-4ff2-9427-a01b64145c6c,3,2,3
6,c74c14bf-d6ae-455d-842c-6c02cd638349,e9ef08e2-9d2d-49d4-8934-7f259cbbf867,2,2,6


In [824]:
pickle.load(open(folder + '/user_id_encoded.pkl', 'rb'))

{'2bd66dd6-5ac6-40ad-968e-1271da13db32': 0,
 'b0af16ed-0633-42a2-afc7-2c5e48db5a48': 1,
 'c74c14bf-d6ae-455d-842c-6c02cd638349': 2}

In [825]:
pickle.load(open(folder + '/farmer_id_encoded.pkl', 'rb'))

{'1000a253-a43c-4ab3-91f7-8c370e01b9c4': 0,
 '2345a62a-8d3a-4f7b-aad5-7e50670a817b': 1,
 '58c5ee91-37b5-4818-8815-872eae86f21c': 2,
 '7488ab52-bc5b-4ff2-9427-a01b64145c6c': 3,
 'a5d3b9fe-eb1f-4c7e-84fb-fd95a39697a7': 4,
 'cecb6e32-8f30-4806-a16c-a6073966ad65': 5,
 'e9ef08e2-9d2d-49d4-8934-7f259cbbf867': 6}

In [826]:
model.save(folder + '/model_collab', save_format='tf')

INFO:tensorflow:Assets written to: ./model/model_collab\assets
